This Notebook will model the rear irradiance for 200 sensors across a 1 panel system, with torquetube, 

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo4'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'New_Panel'

# MakeModule Parameters
moduletype=simulationname
numpanels = 2
x = 1  
y = 2
xgap = 0.01
ygap = 0.0
zgap = 0.02
sensorsy = 200 

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = True
tubetype = 'square'
diameter = 0.20
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=0
numcellsy=0

# SceneDict Parameters
pitch = 4
albedo = 0.62 
clearance_height = 0.5  
nMods = 20
nRows = 7
azimuth_ang=180
tilt =10

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': pitch,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.

for timeindex in range (1421,2160):
    demo.gendaylit(metdata,timeindex)  # Noon, June 17th
    octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
    analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

    frontscan, backscan = analysis.moduleAnalysis(sceneDict['height'], sceneDict['azimuth'], sceneDict['tilt'], 
                                              sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                              moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                              sensorsy=sensorsy)
        
        
    analysis.analysis(octfile, "Shading_"+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance  

path = C:\Users\sayala\Documents\RadianceScenes\Demo4
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'New_Panel')
REWRITING pre-existing module file. 
Module New_Panel successfully created
Missing files, skipping...
Analysis aborted - no octfile 

Adjusting solarposition for sunrise hour, 1422 timeindex
Original datetime 1972-03-01 07:00:00-05:00
Localized and adjusted datetime 1972-03-01 06:50:00-05:00 

Created New_Panel.oct
Linescan in process: Shading_1422_Front
Linescan in process: Shading_1422_Back
Saved: results\irr_Shading_1422.csv
Created New_Panel.oct
Linescan in process: Shading_1423_Front


KeyboardInterrupt: 

In [4]:
moduleDict['moduleoffset']

0